## The Route-Finding Domain

Like all state-space search problems, in a route-finding problem you will be given:
- A start state (for example, `'A'` for the city Arad).
- A goal state (for example, `'B'` for the city Bucharest).
- Actions that can change state (for example, driving from `'A'` to `'S'`).

You will be asked to find:
- A path from the start state, through intermediate states, to the goal state.

We'll use this map:

<img src="../notebook-images/romania.jpg" height="366" width="603">

A state-space search problem can be represented by a *graph*, where the vertices of the graph are the states of the problem (in this case, cities) and the edges of the graph are the actions (in this case, driving along a road).

We'll represent a city by its single initial letter. 
We'll represent the graph of connections as a `dict` that maps each city to a list of the neighboring cities (connected by a road). For now we don't explicitly represent the actions, nor the distances
between cities.

# 1. Search for AIMA 4th edition

Implementation of search algorithms and search problems for AIMA.

We start by defining the abstract class for a `Problem`; problem domains will subclass this, and then you can create individual problems with specific initial states and goals. We also ddefine a `Node` in a search tree, and some functions on nodes: `expand` to generate successors, and `path_actions`, `path_states` and `path` to recover aspects of the path from the node.  Finally, a `PriorityQueue`, which allows you to keep a collection of items, and continually remove from it the item with minimum `f(item)` score.

## 1-1. Import python package &module

In [1]:
import math
import sys

# For some data structure implementation
import heapq
from collections import defaultdict, deque, Counter

## 1-2. Problem abstract class definition

In [2]:
class Problem(object):
    """The abstract class for a formal problem. You should subclass this,
    overriding `actions` and `results`, and other methods if necessary.
    Note: a problem can specify a default heuristic if desired. By default, 
    the heuristic is 0 for all states, and the step cost is 1 for all actions."""

    def __init__(self, initial=None, goal=None, **other_keywords):
        """Specify the initial and goal states.
        Subclasses can use other keywords if they want."""
        self.__dict__.update(initial=initial, goal=goal, **other_keywords) 

    def actions(self, state):           raise NotImplementedError
    def result(self, state, action):    raise NotImplementedError
    def is_goal(self, state):           return state == self.goal
    def step_cost(self, s, action, s1): return 1
    def h(self, node):                  return 0

## 1-3. Tree node definition

In [3]:
class Node:
    "A Node in a search tree."
    def __init__(self, state, parent=None, action=None, path_cost=0):
        self.__dict__.update(state=state, parent=parent, action=action, path_cost=path_cost)

    def __repr__(self): return '<{}>'.format(self.state)
    def __len__(self): return 0 if self.parent is None else (1 + len(self.parent))
    def __lt__(self, other): return self.state < other.state
    
    
def expand(problem, node):
    "Expand a node, generating the children nodes."
    s = node.state
    for action in problem.actions(s):
        s1 = problem.result(s, action)
        cost = node.path_cost + problem.step_cost(s, action, s1)
        yield Node(s1, node, action, cost)
        

def path_actions(node):
    "The sequence of actions to get to this node."
    if node.parent is None:
        return []
    else: 
        return path_actions(node.parent) + [node.action]


def path_states(node):
    "The sequence of states to get to this node."
    if node.parent is None:
        return ([] + [node.state])
    else:
        return (path_states(node.parent)) + [node.state]


def path(node):
    "Alternating states and actions to get to this node."
    if node.parent is None:
        return ([] + [node.state])
    else:
        return (path(node.parent) + [node.action] ) + [node.state]


## 1-4. Queue definition

In [4]:
FIFOQueue = list

LIFOQueue = deque
'''A Collection data structure providing last-in-first-out also called a 'Stack' '''

class PriorityQueue:
    """A queue in which the item with minimum f(item) is always popped first."""

    def __init__(self, items=(), key=lambda x: x): 
        self.key = key
        self.items = []
        for item in items:
            self.add(item)
         
    def add(self, item):
        """Add item to the queuez."""
        heapq.heappush(self.items, (self.key(item), item))

    def pop(self):
        """Pop and return the item with min f(item) value."""
        return heapq.heappop(self.items)[1]

    def __len__(self): return len(self.items)

# 2. Search Algorithms

Here are the six major state-space search algorithms covered in the book:

In [5]:
def breadth_first_search(problem):
    "Search shallowest nodes in the search tree first."
    frontier = LIFOQueue([Node(problem.initial)])
    reached = set()
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached:
                reached.add(s)
                frontier.appendleft(child)
    return failure

def depth_limited_search(problem, limit=5):
    "Search deepest nodes in the search tree first."
    frontier = FIFOQueue([Node(problem.initial)])
    solution = failure
    while frontier:
        node = frontier.pop()
        if len(node) > limit:
            solution = cutoff
        else:
            for child in expand(problem, node):
                if problem.is_goal(child.state):
                    return child
                frontier.append(child)
    return solution

def iterative_deepening_search(problem):
    "Do depth-limited search with increasing depth limits."
    for limit in range(1, sys.maxsize):
        result = depth_limited_search(problem, limit)
        if result != cutoff:
            return result


def best_first_search(problem, f):
    "Search nodes with minimum f(node) value first."
    frontier = PriorityQueue([Node(problem.initial)], key=f)
    reached = {}
    while frontier:
        node = frontier.pop()
        if problem.is_goal(node.state):
            return node
        for child in expand(problem, node):
            s = child.state
            if s not in reached or child.path_cost < reached[s].path_cost:
                reached[s] = child
                frontier.add(child)
    return failure


def uniform_cost_search(problem):
    "Search niodes with minimum  path cost first."
    return best_first_search(problem, lambda node: node.path_cost)


def astar_search(problem, h=None):
    """Search niodes with minimum f(n) = g(n) + h(n)."""
    h = h or problem.h
    return best_first_search(problem, lambda node: node.path_cost + h(node))

failure = Node('failure', path_cost=math.inf) # Indicates an algorithm couldn't find a solution.
cutoff  = Node('cutoff', path_cost=math.inf)  # Indicates iterative deeepening search was cut off.

# 3. Problem Domains

Now we turn our attention to defining some problem domains.

## 3-1. Route Finding Problems

In [14]:
class RouteProblem(Problem):
    """A problem to find a route between places on a map.
    Use RouteProblem('S', 'G', map=Map(...)})"""
    
    def actions(self, state): 
        """The places neighboring `state`. (Action names are same as place names.)"""
        return self.map.neighbors[state]
    
    def result(self, state, action):
        """Go to the `action` place, if the map says that is possible."""
        return action if action in self.map.neighbors[state] else state
    
    def step_cost(self, s, action, s1):
        """The actual distance between s and s1."""
        return self.map.distances[s, s1]
    
    def h(self, node):
        "Straight-line distance between state and the goal."
        locs = self.map.locations
        s, g = locs[node.state], locs[self.goal]
        return abs(complex(*s) - complex(*g))

class Map:
    """Builds an undirected graph of {vertex: [neighbors...]}, with two additional annotations:
       neighbors:
       distances: a dict of {(v1, v2): number} giving the distance from v1 to v2;
       locations: a dict of {v: (x, y)} giving the (x, y) location of each vertex."""
    def __init__(self, distances, locations=()):
        self.neighbors = undirected_graph(distances)
        self.distances = distances
        self.locations = locations or defaultdict(lambda: (0, 0))
        for (v1, v2) in list(distances):
            distances[v2, v1] = distances[v1, v2]
            
def undirected_graph(pairs):
    "Given {(v1, v2)...} pairs, return a graph of {v1: [v2,...], v2:[v1,...]}."
    graph = defaultdict(tuple)
    for (v1, v2) in pairs:
        graph[v1] += (v2,)
        graph[v2] += (v1,)
    return dict(graph)

romania = Map(distances={
    ('O', 'Z'): 71, ('O', 'S'): 151, ('A', 'Z'): 75, ('A', 'S'): 140, ('A', 'T'): 118, 
    ('L', 'T'): 111, ('L', 'M'): 70, ('D', 'M'): 75, ('C', 'D'): 120, ('C', 'R'): 146, 
    ('C', 'P'): 138, ('R', 'S'): 80, ('F', 'S'): 99, ('B', 'F'): 211, ('B', 'P'): 101, 
    ('B', 'G'): 90, ('B', 'U'): 85, ('H', 'U'): 98,  ('E', 'H'): 86, ('U', 'V'): 142, 
    ('I', 'V'): 92, ('I', 'N'): 87, ('P', 'R'): 97},
    locations=dict(
    A=(91, 492), B=(400, 327), C=(253, 288), D=(165, 299), E=(562, 293), F=(305, 449),
    G=(375, 270), H=(534, 350), I=(473, 506), L=(165, 379), M=(168, 339), N=(406, 537),
    O=(131, 571), P=(320, 368), R=(233, 410), S=(207, 457), T=(94, 410), U=(456, 350),
    V=(509, 444), Z=(108, 531)))

In [20]:
romania.neighbors['A'] # Neighbors of

('Z', 'S', 'T')

In [21]:
romania.distances['A', 'Z']

75

In [17]:
romania.locations['A']

(91, 492)

<img src="../notebook-images/romania.jpg" height="366" width="603">

In [27]:
# Solve a problem (which gives a node) and recover the sequence of states in that node's path
problem_r1 = RouteProblem('D', 'O', map=romania)
# Breadth first search finds a solution with fewer steps, but in this case higher path cost
path_states(breadth_first_search(problem_r1))

['D', 'C', 'R', 'S', 'O']

## 3-2. Water Pouring Problems

In [13]:
class PourProblem(Problem):
    """Problem about pouring water between jugs to achieve some water level.
    Each state is a tuples of water levels. In the initialization, provide a tuple of 
    sizes, e.g. PourProblem((2, 4, 3), 7, sizes=(8, 16, 32)), 
    which means three jugs of sizes (8, 16, 32), initially filled with (2, 4, 3) units of 
    water, respectively, and the goal is to get a level of 7 in any one of the jugs."""
    
    def actions(self, state):
        """The actions executable in this state."""
        jugs = range(len(state))
        return ([('Fill', i)    for i in jugs if state[i] < self.sizes[i]] +
                [('Dump', i)    for i in jugs if state[i]] +
                [('Pour', i, j) for i in jugs if state[i] for j in jugs if i != j])

    def result(self, state, action):
        """The state that results from executing this action in this state."""
        result = list(state)
        act, i, *_ = action
        if act == 'Fill':   # Fill i to capacity
            result[i] = self.sizes[i]
        elif act == 'Dump': # Empty i
            result[i] = 0
        elif act == 'Pour': # Pour from i into j
            j = action[2]
            amount = min(state[i], self.sizes[j] - state[j])
            result[i] -= amount
            result[j] += amount
        return tuple(result)

    def is_goal(self, state):
        """True if the goal level is in any one of the jugs."""
        return self.goal in state
    
    
class GreenPourProblem(PourProblem): 
    """A PourProblem in which we count not the number of steps, but the amount of water used."""
    def step_cost(self, state, action, result=None):
        "The cost is the amount of water used in a fill."
        act, i, *_ = action
        return self.sizes[i] - state[i] if act == 'Fill' else 0

# Reporting Metrics

Now let's gather some metrics on how well each algorithm does.  We'll use `CountCalls` to wrap a `Problem` object in such a way that calls to its methods are delegated, but each call increments a counter. Once we've solved the problem, we print out summary statistics.

In [ ]:
class CountCalls:
    """Delegate all attribute accesses to the object, and count them in ._counts"""
    def __init__(self, obj):
        self._object = obj
        self._counts = Counter()
        
    def __getattr__(self, attr):
        self._counts[attr] += 1
        return getattr(self._object, attr)
        
def report(searchers, problems):
    "Show metrics for each searcher on each problem."
    for searcher in searchers:
        print(searcher.__name__ + ':')
        total_counts = Counter()
        for p in problems:
            prob = CountCalls(p)
            soln = searcher(prob)
            cts  = prob._counts; 
            cts.update(len=len(path_actions(soln)), cost=soln.path_cost)
            total_counts += cts
            report_line(cts, type(p).__name__)
        report_line(total_counts, 'TOTAL\n')
        
def report_line(counts, name):
    "Print one line of the report."
    print('{:7,d} Exp |{:7,d} Gen |{:7,d} Goal |{:5.0f} cost |{:3d} len | {}'
          .format(counts['actions'], counts['result'], counts['is_goal'], 
                  counts['cost'], counts['len'], name))

In [ ]:
# Here's a tiny report
report([astar_search], [p1, p2])

The last line says that, over the two problems `[p1, p2]`, the `astar_search` algorithm expanded 528 nodes, generating 4,706 nodes and doing 530 goal checks. Together, the two solutions had a path cost of 13 and also a total length of 13 (since step cost was 1 in these problems). 

Now let's do a bigger report, concentrating first on the easier problems, then harder ones:

In [ ]:
easy = (p1, g1, r1, r2, r3, r4, e1)
hard = (g2, p2, g3, p3, g4, p4, e2, e3, e4)

report((astar_search, uniform_cost_search,  breadth_first_search, 
        iterative_deepening_search, depth_limited_search), easy)

One thing to notice: on three of the problems, `depth_limited_search` had a path cost of `inf`, meaning that the search was cut off, so it reported an infinite cost.

If we look at the whole `cost` column, we see that the optimal algorithms, `astar_search` and `uniform_cost_search`, give the best results, while `breadth_first_search` and `iterative_deepening_search` have non-optimal costs on some problems, because they find a solution with the minimal number of steps, but not the minimal path cost. We see that `astar_search` has fewer expansions, generated nodes, and goal tests that `uniform_cost_search`, which means the heuristic helps (if only by 10% or so).

Next I'll try some harder problems; I won't even try the tree search algorithms on these problems; too many redundant paths.

In [ ]:
report((astar_search, uniform_cost_search, breadth_first_search), hard)

This time we see that A* is an order of magnitude more efficient than the two uninformed algorithm. Note that again,  uniform cost is optimal, but breadth-first is not: it optimized for path length, not path cost.